# Muestreo de datos con una ventana deslizante

El  siguiente  paso,  antes  de  poder  crear  las  incrustaciones  para  el  LLM,  es  generar  los  pares  de  entradadestino  necesarios  para  entrenar  un  LLM.

![Texto alternativo](./imgs/2.10.png)

Dada  una  muestra  de  texto,  se  extraen  bloques  de  entrada  como  submuestras  que  sirven  como  entrada  para  el  LLM.  La  tarea  de  predicción  del  LLM  durante  el  entrenamiento  consiste  en  predecir  la  siguiente  palabra  que  sigue  al  bloque  de  entrada.  Durante  el  entrenamiento,  se  enmascaran  todas  las  palabras  que  superan  el  objetivo. 

In [14]:
#Para comrnzar hayr que abrir el documento de verdicts y tokenizarlo
import tiktoken

with open("../txt/The_Verdict.txt", "r", encoding="utf-8") as f:
    r_text = f.read()

tokenizer = tiktoken.get_encoding('gpt2')

ids_text = tokenizer.encode(r_text)
print(len(ids_text))

5147


In [15]:
#Eliminar los 50 priemros tokens del conjunto para hacer un demostración
ids_demostracion = ids_text[50:]

#  Una  de  las  formas  más  fáciles  e  intuitivas  de  crear  los  pares  de  entradaobjetivo  para  la  tarea  de  predicción  de  la  siguiente  
# palabra  es  crear  dos  variables,  x  e  y,  donde  x  contiene  los  tokens  de  entrada  e  y  contiene  los  objetivos,  que  son  las  entradas  
# desplazadas  en 1.

tam = 4 #num de tokens por entrada
x = ids_demostracion[:tam]
y = ids_demostracion[1:tam+1]
print(x)
print(y)

[290, 4920, 2241, 287]
[4920, 2241, 287, 257]


Al  procesar  las  entradas  junto  con  los  objetivos,  que  son  las  entradas  desplazadas  una  posición,  podemos  crear  las  tareas  de  predicción  de  la  siguiente  palabra

In [16]:
for i in range(1, tam+1):
    context = ids_demostracion[:i]
    pred = ids_demostracion[i]
    print("Contexto: ", context, "Predicicón: ", pred)

Contexto:  [290] Predicicón:  4920
Contexto:  [290, 4920] Predicicón:  2241
Contexto:  [290, 4920, 2241] Predicicón:  287
Contexto:  [290, 4920, 2241, 287] Predicicón:  257


In [17]:
#Ahora con fin ilsutrativos se muestra convirtiendo lostokens en texto
for i in range(1, tam+1):
    context = ids_demostracion[:i]
    pred = ids_demostracion[i]
    print("Contexto: ", tokenizer.decode(context), "Predicción: ", tokenizer.decode([pred]))


Contexto:   and Predicción:   established
Contexto:   and established Predicción:   himself
Contexto:   and established himself Predicción:   in
Contexto:   and established himself in Predicción:   a


Solo  queda  una  tarea  más  antes  de  que  podamos  convertir  los  tokens  en  incrustaciones. Implementar  un  cargador  de  datos  eficiente  que  itere  sobre  el  conjunto  de  datos  de  entrada  y  devuelva  las  entradas  y  los  destinos  como  tensores  de  PyTorch,  que  pueden  considerarse  como  matrices  multidimensionales.

![Texto alternativo](./imgs/2.11.png)

Para  implementar  cargadores  de  datos  eficientes,  recopilamos  las  entradas  en  un  tensor,  x,  donde  cada  fila  representa  un  contexto  de  entrada.  Un  segundo  tensor,  y,  contiene  los  objetivos  de  predicción  correspondientes  (palabras siguientes),  que  se  crean  desplazando  la  entrada  una  posición.

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_lenght, stride):
        self.input_ids = []
        self.labels_ids = []

        tokens_ids = tokenizer.encode(txt) #Tokenzar el texto completo

        for i in range(0, len(tokens_ids) - max_lenght, stride): #Uso de una  ventana  deslizante  para  dividir  el  libro  en  secuencias  superpuestas  de  longitud  máxima
            self.input_ids.append(torch.tensor(tokens_ids[i:i+max_lenght]))
            self.labels_ids.append(torch.tensor(tokens_ids[i+1:i+max_lenght+1]))

    def __len__(self): #Devuelve  el  número  total  de  filas  en  el  conjunto  de  datos
        return len(self.labels_ids)
    
    def __getitem__(self, index): #Devuelve  una  sola  fila  del  conjunto  de  datos
        return self.input_ids[index], self.labels_ids[index] 

La  clase  GPTDatasetV1  del  listado se  basa  en  la  clase  Dataset  de  PyTorch  y  define  cómo  se  obtienen  las  filas  individuales  del  conjunto  de  datos.  Cada  fila  consta  de  un  número  de  identificadores  de  token  (basados  en  una  longitud  máxima)  asignados  a  un  tensor.

In [34]:
#Creacion de los dataloaders(cargadores de datos)
def create_dataloader_v1(txt, batch_size=4, max_length=256,
       stride=128, shuffle=True, drop_last=True, num_workers=0):
   tokenizer = tiktoken.get_encoding("gpt2")         #inicializar toenizer          
   dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)    #Crear dataset
   dataloader = DataLoader( 
       dataset,
       batch_size=batch_size,
       shuffle=shuffle,
       drop_last=drop_last,         #descarta  el  último  lote  si  es  más  corto  que  el  tamaño  de  lote  especificado                          
       num_workers=num_workers               #La  cantidad  de  procesos  de  CPU                         
   )
   return dataloader

#probar el cargador de daros con un tamaño de lote de 1 para una LLM con un tamaño de contexto de 4.

dl = create_dataloader_v1(r_text, batch_size=2, max_length=8, stride=2, shuffle=False)
data_iter = iter(dl)
fr_batch = next(data_iter)
print(fr_batch)
sc_batch = next(data_iter)
print(sc_batch)

[tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271],
        [ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899],
        [ 1464,  1807,  3619,   402,   271, 10899,  2138,   257]])]
[tensor([[ 1807,  3619,   402,   271, 10899,  2138,   257,  7026],
        [  402,   271, 10899,  2138,   257,  7026, 15632,   438]]), tensor([[ 3619,   402,   271, 10899,  2138,   257,  7026, 15632],
        [  271, 10899,  2138,   257,  7026, 15632,   438,  2016]])]


Si  comparamos  el  primer  lote  con  el  segundo,  observamos  que  los  ID  de  token  del  segundo  lote  se  desplazan  una  posición  con  respecto  al  primero  (por  ejemplo,  el  segundo  ID  en  la  entrada  del  primer  lote  es  367,  que  es  el  primer  ID  de  la  entrada  del  segundo  lote).  La  configuración  de  paso  determina  el  número  de  posiciones  que  se  desplazan  las  entradas  entre  lotes,  emulando  un  enfoque  de  ventana  deslizante

![Texto alternativo](./imgs/2.12.png)

In [36]:
dataloader = create_dataloader_v1(r_text, batch_size=8, max_length=4, stride=4)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[ 2119,    13,   198,   198],
        [   12, 12239,    13,   198],
        [  402,   271, 10899,   338],
        [  319,   257,  1308, 27461],
        [  257,  8212,   326, 13663],
        [ 1813,   340,   284,   502],
        [   11,   355,  9074,    13],
        [12375,   299, 20896,    82]])

Targets:
 tensor([[   13,   198,   198,     1],
        [12239,    13,   198,   198],
        [  271, 10899,   338, 12036],
        [  257,  1308, 27461,  1171],
        [ 8212,   326, 13663,   262],
        [  340,   284,   502,    11],
        [  355,  9074,    13,   536],
        [  299, 20896,    82, 24357]])


Tenga  en  cuenta  que  aumentamos  el  paso  a  4.  Esto  es  para  utilizar  el  conjunto  de  datos  en  su  totalidad  (no  omitimos  ni  una  sola  palabra)  pero  también  para  evitar  cualquier  superposición  entre  los  lotes,  ya  que  una  mayor  superposición  podría  llevar  a  un  mayor  sobreajuste.

[Agregar tokens de contexto especiales](./6_incrustaciones_tokens.ipynb)